# Convolutional neural network

In this section, we will extend the simple neural network framework to work with convolutional neural network (CNN).

In [ ]:
import numpy as np
from narray import set_library

xp = set_library('numpy')

%matplotlib inline

In [ ]:
import nn
import neurons
from dataset import MNIST

train_data = MNIST('data/mnist/train-images-idx3-ubyte.gz', 'data/mnist/train-labels-idx1-ubyte.gz')
test_data = MNIST('data/mnist/t10k-images-idx3-ubyte.gz', 'data/mnist/t10k-labels-idx1-ubyte.gz')

In [ ]:
num_examples = train_data.sample_size
num_inputs = np.prod(train_data.input_dim)
num_outputs = np.prod(train_data.target_dim)

In [ ]:
scale_weight = .01
batch_size = 64

data_layer = nn.BatchDataLayer(train_data.inputs, train_data.targets, batch_size, shuffle=True)

conv_layer1 = nn.ConvolutionalLayer(
    filter_size=(3,3), stride=(1,1), pad=(1,1),
    num_filters=10, scale=scale_weight, neuron_type=neurons.ReLU)
pool_layer1 = nn.PoolingLayer(
    filter_size=(2,2), stride=(2,2), pad=(0,0))

conv_layer2 = nn.ConvolutionalLayer(
    filter_size=(5,5), stride=(1,1), pad=(2,2),
    num_filters=20, scale=scale_weight, neuron_type=neurons.ReLU)
pool_layer2 = nn.PoolingLayer(
    filter_size=(2,2), stride=(2,2), pad=(0,0))

fc_layer1 = nn.FullyConnectedLayer(
    num_neurons=128, scale=scale_weight, neuron_type=neurons.ReLU)
fc_layer2 = nn.FullyConnectedLayer(
    num_neurons=10, scale=scale_weight)

softmax_layer = nn.SoftmaxLayer()
loss_layer = nn.CrossEntropyLayer()

cnn_net = nn.NeuralNetwork(
    data_layer,
    [
     conv_layer1,
     pool_layer1,
#      conv_layer2,
#      pool_layer2,
     fc_layer1,
     fc_layer2,
     softmax_layer
    ],
    loss_layer)


In [ ]:
from timeit import default_timer as timer

epochs = 10
learning_rate = .002
num_batches = 50

for e in range(epochs):
    print("--- Epoch %d ---" % e)
    start = timer()
    loss = cnn_net.train(learning_rate, num_batches, verbose=False)
    end = timer()
    # print("param", softmax_layer.parameters())
    # print("grad", softmax_layer.gradient())
    print("loss %f (%.3f seconds elapsed)" % (loss, end - start))

In [ ]:
def model_predict(net, inputs, labels):
    yhat = net.evaluate(inputs)
    pred_labels = np.argmax(yhat, axis=1)
    num_correct = np.sum(pred_labels == labels)
    return num_correct, pred_labels

In [ ]:
total_num_correct = 0
predicted_labels = np.array([])

for batch_start in range(0, batch_size * 10, batch_size):
    input_batch = test_data.inputs[batch_start:batch_start+batch_size]
    label_batch = test_data.labels[batch_start:batch_start+batch_size]
    num_correct, labels = model_predict(cnn_net, input_batch, label_batch)
    total_num_correct += num_correct
    predicted_labels = np.append(predicted_labels, labels, axis=0)

print("%.2f%% of %d" % (100 * total_num_correct / len(predicted_labels), len(predicted_labels)))

In [ ]:
for i in range(5):
    test_data.show_image(i, "prediction %d vs target %d" % (predicted_labels[i], test_data.labels[i]))